In [ ]:
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [ ]:
# Preprocess the data
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
# Define the model architecture
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dense(10, activation='softmax'))

In [ ]:
# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(x_test, y_test))

Epoch 1/10
469/469 [==============================] - 15s 28ms/step - loss: 0.2582 - accuracy: 0.9254 - val_loss: 0.1256 - val_accuracy: 0.9620
Epoch 2/10
469/469 [==============================] - 5s 11ms/step - loss: 0.1048 - accuracy: 0.9689 - val_loss: 0.0985 - val_accuracy: 0.9692
Epoch 3/10
469/469 [==============================] - 5s 10ms/step - loss: 0.0687 - accuracy: 0.9794 - val_loss: 0.0775 - val_accuracy: 0.9768
Epoch 4/10
469/469 [==============================] - 5s 10ms/step - loss: 0.0499 - accuracy: 0.9848 - val_loss: 0.0707 - val_accuracy: 0.9784
Epoch 5/10
469/469 [==============================] - 4s 10ms/step - loss: 0.0369 - accuracy: 0.9891 - val_loss: 0.0673 - val_accuracy: 0.9789
Epoch 6/10
469/469 [==============================] - 4s 9ms/step - loss: 0.0284 - accuracy: 0.9919 - val_loss: 0.0637 - val_accuracy: 0.9825
Epoch 7/10
469/469 [==============================] - 4s 10ms/step - loss: 0.0220 - accuracy: 0.9935 - val_loss: 0.0616 - val_accuracy: 0.9826

In [ ]:
# Evaluate the model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.06537145376205444
Test accuracy: 0.9812999963760376


# Long short time memory Neural Net

model Long-Short-Term Memory in python. Include dataset, training, testing and metrics



In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import requests

In [2]:
URL = "https://data.mendeley.com/public-files/datasets/jtv3f9pbdb/files/95057ed1-dd44-4403-98b7-cb436215cc1f/file_downloaded"
data = requests.get(URL)

In [3]:
# load and preprocess the data
data = pd.read_csv("data.csv")
data = data.dropna() # remove missing values
data = data.values
data = data.astype("float32")

FileNotFoundError: ignored

In [ ]:
# scale the data between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)


In [ ]:
# split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)


In [ ]:
# convert the data into a 3D array (sample, timestep, feature)
def create_dataset(data, look_back=1):
    X, y = [], []
    for i in range(len(data) - look_back - 1):
        a = data[i:(i + look_back), :]
        X.append(a)
        y.append(data[i + look_back, :])
    return np.array(X), np.array(y)

look_back = 1
X_train, y_train = create_dataset(train_data, look_back)
X_test, y_test = create_dataset(test_data, look_back)


In [ ]:
# reshape the data into a 3D array (samples, timesteps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

In [ ]:
# build and compile the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(look_back, X_train.shape[2])))
model.add(Dense(X_train.shape[2]))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

In [ ]:
# make predictions on the test set
y_pred = model.predict(X_test)

In [ ]:
# scale the predictions back to the original range
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test)

In [ ]:
# evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)